In [1]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error,r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pickle

import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.model_selection import train_test_split , StratifiedShuffleSplit , RandomizedSearchCV
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('Dataset/cars_data.csv')
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [4]:
df1 = df.drop(columns='Car_Name')
df1.shape

(301, 8)

In [5]:
# add column of how old vehicle 
# Find today's date for check that how much car is old

x = datetime.datetime.now()
x.year

df1['how_old'] = x.year - df1.Year
df1.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,how_old
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0,9
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0,10
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0,6
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0,12
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0,9


In [6]:
# data manipulation - categorical features into the numeric features; applied OneHotEncoding

df1.drop(columns = 'Year' , axis=1 , inplace=True)

df1 = pd.get_dummies(df1 , drop_first=True)
df1.head()      

,Selling_Price,Present_Price,Kms_Driven,Owner,how_old,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Individual,Transmission_Manual
0,3.35,5.59,27000,0,9,False,True,False,True
1,4.75,9.54,43000,0,10,True,False,False,True
2,7.25,9.85,6900,0,6,False,True,False,True
3,2.85,4.15,5200,0,12,False,True,False,True
4,4.60,6.87,42450,0,9,True,False,False,True


In [7]:
# divide data into dependent and independent features

X = df1.drop(columns='Selling_Price')
y = df1['Selling_Price']

# Split data into training and testing

shuffle = StratifiedShuffleSplit(random_state=51 , test_size=0.2 , n_splits=1)

for train_index , test_index in shuffle.split(df1 , df1['Fuel_Type_Diesel'] , df1['Fuel_Type_Petrol']):
    X_train_shuffle = df1.iloc[train_index]
    X_test_shuffle =  df1.iloc[test_index]
    
X_train_shuffle.shape ,  X_test_shuffle.shape

((240, 9), (61, 9))

In [8]:
X_train = X_train_shuffle.drop(columns = 'Selling_Price')
y_train = X_train_shuffle['Selling_Price']
X_test = X_test_shuffle.drop(columns = 'Selling_Price')
y_test = X_test_shuffle['Selling_Price']
X_train.shape , X_test.shape , y_train.shape , y_test.shape

((240, 8), (61, 8), (240,), (61,))

In [13]:
pickled_model = pickle.load(open('model/car_price_prediction_model.pkl', 'rb'))  # without PCA

In [14]:
pickled_model.predict(X_test)

array([ 9.88855593,  0.48133734,  0.69450269,  0.39109805, 16.38743333,
        2.9846713 ,  3.13696574,  5.76357917,  4.22462213,  6.98697143,
        5.30277951,  5.65352054,  0.36565099,  6.78921795,  3.12921574,
        8.03531667, 10.26508945,  1.41607571,  5.167643  ,  5.11960488,
        1.28176981, 20.89134167,  4.3058166 ,  7.38270152,  6.19327222,
        3.50983238,  3.21891683,  1.14729295,  2.55145702,  5.31462278,
        5.72231783,  1.81382611, 20.49368333,  2.52295972,  4.33570855,
        5.03725919,  9.96711526,  0.46671128,  5.39605615, 15.91726667,
        7.33390228,  0.49548329,  6.98318742,  5.38490181,  3.2216775 ,
        0.4609661 ,  5.89417361,  5.33016938,  5.6021441 ,  0.13353245,
        3.03748935,  1.14368467,  2.66056944,  3.62224095,  2.79832796,
        5.09012165,  9.98193052,  5.92173694, 10.29254257,  0.31831566,
        3.17034491])

In [15]:
pred=pickled_model.predict(X_test)
price=np.exp(pred[0])
print(price)

19703.586084948114
